This notebook is dedicated to the application of SVD and subsequently feature extraction and clustering to the various datasets prepared in the data collection & cleaning notebook. 

In [ ]:
#import necessary packages
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import seaborn as sns

In [ ]:
from numpy import linalg as LA

Below are functions that will be used throughout the notebook. Their use is in the comments & will become more clear with application.

In [ ]:
#this function is used to calculate the "knee" of a given function. It's application in this program is to calculate
#the optimal value of k when performing truncated SVD
def calculateKnee(values):
    #get coordinates of all the points
    nPoints = len(values)
    allCoord = np.vstack((range(nPoints), values)).T
    #np.array([range(nPoints), values])

    # get the first point
    firstPoint = allCoord[0]
    # get vector between first and last point - this is the line
    lineVec = allCoord[-1] - allCoord[0]
    lineVecNorm = lineVec / np.sqrt(np.sum(lineVec**2))

    # find the distance from each point to the line:
    # vector between all points and first point
    vecFromFirst = allCoord - firstPoint

    # To calculate the distance to the line, we split vecFromFirst into two 
    # components, one that is parallel to the line and one that is perpendicular 
    # Then, we take the norm of the part that is perpendicular to the line and 
    # get the distance.
    # We find the vector parallel to the line by projecting vecFromFirst onto 
    # the line. The perpendicular vector is vecFromFirst - vecFromFirstParallel
    # We project vecFromFirst by taking the scalar product of the vector with 
    # the unit vector that points in the direction of the line (this gives us 
    # the length of the projection of vecFromFirst onto the line). If we 
    # multiply the scalar product by the unit vector, we have vecFromFirstParallel
    scalarProduct = np.sum(vecFromFirst * np.matlib.repmat(lineVecNorm, nPoints, 1), axis=1)
    vecFromFirstParallel = np.outer(scalarProduct, lineVecNorm)
    vecToLine = vecFromFirst - vecFromFirstParallel

    # distance to line is the norm of vecToLine
    distToLine = np.sqrt(np.sum(vecToLine ** 2, axis=1))

    # knee/elbow is the point with max distance value
    idxOfBestPoint = np.argmax(distToLine)

    print("Knee of the curve is at index =",idxOfBestPoint)
    return idxOfBestPoint

#### Master Matrix

First we will apply SVD to our large dataframe - titled Master Dataframe. 

In [ ]:
#load in clean csv file 
master_df = pd.read_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/master_df.csv")

### Applying SVD

Prior to applying truncated SVD, we will perform traditional svd to extract the singular values to determine the optimal rank to use when performced truncated SVD.

In [ ]:
#convert pandas dataframe objects to numpy arrays
master_matrix = master_df.values
#apply numpy SVD function to master matrix
master_U,master_s,master_V = np.linalg.svd(master_matrix,full_matrices = False)

# Interpreting SVD Results

The size of each singular values (entries in s) tells you how much of the dataset's total variance is accounted for by each singular vector.

The results of SVD can give us a matrix with lower rank that is said to approximate the original matrix. To reconstruct a lower dimensional matrix we must first select the top k largest singular values in s.

### Selecting K 

* Plot singular values using a scree plot. A suitable cutoff point k is where the slope of the plot appears to flatten or where there is a detectable elbow or knee in the curve.
* Calculate the contribution of each singular value by calculating the sum $f_k$ and the entropy of the dataset. The magnitude of the entropy indicates how many dimensions need to be retained 

##### 1. Plot Singular Values with a Scree Plot

A scree plot is simply a scatter plot in which the magnitudes of the singular values are plotted in order.

In [ ]:
plt.plot(range(0,93),master_s)
plt.title('Scree Plot of Singular Values: Master Matrix')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')

In [ ]:
calculateKnee(master_s)

##### 2. Calculate contribution of each value using $f_k$ and the entropy of the dataset. 

$f_k = s_k^2/\sum_{i=1}^rs_i^2$ and $entropy = -1/\log(r)\sum_{k=1}^rf_k\log(f_k)$

In [ ]:
#declare empty list to contain f_k values
f_k_list = []
s_2 = []

for value in master_s:
    s_2.append(value**2)

s_i_sum = np.sum(s_2)
for i in range(69):
    s_k2 = s[i]**2
    f_k = s_k2/s_i_sum
    f_k_list.append(f_k)
    
log = -1*math.log(69,10)
print(log)
f_k_sum = 0
for i in range(69):
    f_k = f_k_list[i] 
    log_value = math.log(f_k,10)
    f_k_sum = f_k * log_value

print(log)
print(f_k_sum)
print("F_k values: ",f_k_list)
print("Entropy: ", entropy)

### Truncated SVD

Given the calculated k, we will perform truncated svd to produce an approximate master matrix with k features.

We can use sklearn's TruncatedSVD function to compute the SVD and compute the new approximated matrix. We can use this matrix to perform clustering algorithms in a lower dimensional space.

In [ ]:
svd = TruncatedSVD(n_components = 7)
svd.fit(master_matrix)
#Fit LSI model on training data X
truncated_mmSVD = svd.transform(master_matrix)
#Fit LSI model to X and perform dimensionality reduction on X.
reducedTrunc_mmSVD = svd.fit_transform(master_matrix)

### Model Evaluation

Sklearn's truncatedSVD function has built a built in explained variance ratio to evaluate model performance. In addition, we will also calculate the frobenius norm of the diffference between the original and approximate matrices(cite wherever I learned that this is something I might want to do)

In [ ]:
print("The explained variance for the original LSI model is: ", truncated_mmSVD.explained_variance_ratio)
print("The explained variance for the model with dimensionality reduction is: " reducedTrunc_mmSVD.explained_variance_ratio)

In [ ]:
mm_approxDiff = np.subtract(master_matrix, truncated_mmSVD)
mm_redApproxDiff = np.subtract(master_matrix, reducedTrun_mmSVD)

print("The frobenius norm of the difference is: ", LA.norm(mm_approxDiff))
print("The frobenius norm of the reduced difference is: " LA.norm(mm_redApproxDiff))

#  Lab Data

Given previous results, it is clear that far less features than available in the master dataframe are necessary to get a clear snapshot of the data. To narrow the scope of my analysis, I chose to perform a targeted analysis of first day lab data and mortality outcomes. The SVD computations performed above were condensed into easy to use functions to save time.

Additionally, I performed truncatedSVD on standardized lab data that was later used for SDD calculations.

In [ ]:
def calculateK(A):
    #compute basic svd to find K value
    U,s,V = np.linalg.svd(A,full_matrices)
    k = calculateKnee(s)
    return k

In [ ]:
#load in clean csv file
lab_data = pd.read_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/lab_data.csv",low_memory = True)
standardizedLab_df = pd.read_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/standardizedLab_df.csv", low_memory = True)

In [ ]:
#store dataframe values in numpy arrays
lab_matrix = lab_data.values
standardizedLabMatrix = standardizedLab_df.values

In [ ]:
#apply numpys SVD function (reduced version due to memory constraints)
lab_k = calculateK(lab_matrix)
standLab_k = calculatedK(standardizedLabMatrix)

Given the calculated values of k, a truncated SVD will be performed in a similar fashion to how it was done with the master data.

In [ ]:
labSVD = TruncatedSVD(n_components = lab_k)
labSVD.fit(lab_matrix)
#Fit LSI model on training data X
truncated_labSVD = labSVD.transform(lab_matrix)
#Fit LSI model to X and perform dimensionality reduction on X.
reducedTrunc_labSVD = labSVD.fit_transform(lab_matrix)

In [ ]:
standLabSVD = TruncatedSVD(n_components = standLab_k)
standLabSVD.fit(standardizedLabMatrix)
#Fit LSI model on training data X
truncated_standLabSVD = standLabSVD.transform(standardizedLabMatrix)
#Fit LSI model to X and perform dimensionality reduction on X.
reducedTrunc_standLabSVD = standLabSVD.fit_transform(standardizedLabMatrix)

### Model Evaluation

In [ ]:
print("The explained variance for the lab data's  original LSI model is: ", truncated_labSVD.explained_variance_ratio)
print("The explained variance for the lab data's model with dimensionality reduction is: " 
      reducedTrunc_LabSVD.explained_variance_ratio)

print("The explained variance for the standardized lab data's  original LSI model is: ", 
      truncated_standLabSVD.explained_variance_ratio)
print("The explained variance for the standardized lab data's model with dimensionality reduction is: " 
      reducedTrunc_standLabSVD.explained_variance_ratio)

In [ ]:
lab_approxDiff = np.subtract(lab_matrix, truncated_labSVD)
lab_redApproxDiff = np.subtract(lab_matrix, reducedTrun_labSVD)

print("The frobenius norm of the difference is: ", LA.norm(lab_approxDiff))
print("The frobenius norm of the reduced difference is: " LA.norm(lab_redApproxDiff))

standLab_approxDiff = np.subtract(standardizedLabMatrix, truncated_standLabSVD)
standLab_redApproxDiff = np.subtract(standardizedLabMatrix, reducedTrun_standLabSVD)

print("The frobenius norm of the difference is: ", LA.norm(standLab_approxDiff))
print("The frobenius norm of the reduced difference is: " LA.norm(standLab_redApproxDiff))

Following these results, I decided to use my results to apply the k-means clustering algorithm tos ee if any further groupings in the data could be detected. 

### Clustering

K-means clustering was performed using sklearn's built in clustering functions. Optimal values for k were chosen given a variety of documented "tests".

In [ ]:
#import Kmeans package from sklearn 
from sklearn.cluster import KMeans 

In [ ]:
#generate elbow plot to detect optimal number of clusters 
from scipy import cluster
cluster_array = [cluster.vq.kmeans(results, i) for i in range(1,10)]

plt.plot([var for (cent,var) in cluster_array])
plt.show()

In [ ]:
#convert lab results to be stored in a pandas dataframe
truncatedLab_df = pd.DataFrame(truncated_LabSVD)

In [ ]:
#K means Clustering method using sklearn functions 
def doKmeans(X, nclust=2):
    model = KMeans(nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    return (clust_labels, cent)

clust_labels, cent = doKmeans(truncatedLab_df, 2)
kmeans = pd.DataFrame(clust_labels)
truncatedLab_df.insert((truncatedLab_df.shape[1]),'kmeans',kmeans)

In [ ]:
#Plot the clusters obtained using k means
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(truncatedLab_df[0],truncatedLab_df[1],
                     c=kmeans[0],s=50)
plt.colorbar(scatter)
plt.show()

In [ ]:
nmf_master_df = pd.read_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/nmf_master_df.to_csv", low_memory = True)

In [ ]:
nmf_lab_data = lab_data.apply(lambda x: x.fillna(0))

In [ ]:
size = 22
corr = lab_data.corr()
fig, ax = plt.subplots(figsize=(size, size))
ax.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns);
plt.yticks(range(len(corr.columns)), corr.columns);

In [ ]:
rs = np.random.RandomState(0)
df = lab_data
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
standardizedLabMatrix = standardizedLab_df.values

In [ ]:
#apply numpys SVD function (reduced version due to memory constraints)
redLab_U, redLab_s, redLab_v = np.linalg.svd(standardizedLabMatrix,full_matrices = False)

In [ ]:
sddLab_U, sddLab_s, sddLab_v = np.linalg.svd(sdd_labMatrix,full_matrices = False)

In [ ]:
np.savetxt("C:/Users/rachh/Downloads/sddLabMatrix.csv", results, delimiter=",")

Master Matrix Plotting

Lab Data Plotting

## Approximate matrix B for original matrix 

Export approximated matrix and store in .csv file

In [ ]:
matrix_transformed = svd.fit_transform(master_matrix)

Lab Clean Data Truncated SVD

In [ ]:
lab_svd = TruncatedSVD(n_components = 5)
lab_svd.fit(standardizedLab_matrix)
results = lab_svd.transform(standardizedLab_matrix)
print(results)

In [ ]:
sdd_lab_svd = TruncatedSVD(n_components = 2)
sdd_lab_svd.fit(sdd_labMatrix)
sddLab_results = sdd_lab_svd.transform(sdd_labMatrix)

In [ ]:
np.savetxt("C:/Users/rachh/Downloads/sdd_lab_svd.csv",sddLab_results)

In [ ]:
#run SDD on truncated matrix produced by SVD - the logic behind this is that the SVD is denoising the matrix so that the 
#SVD can better detect the structure within it 
#labels from early columns of X can be used as labels for the clusters that the SDD finds 

In [ ]:
#for nmf we don't want to replace missing values, can accept sparse matrices as input 

### Implementing NMF

Now we will implement the NMF algorithms on the respective datasets 

In [ ]:
from sklearn.decomposition import NMF

Determine number of components for NMF lab data

In [ ]:
lab_matrix = lab_data.values
nmf_lab_matrix = nmf_lab_data.values
U,s,V = np.linalg.svd(nmf_lab_matrix,full_matrices = False)

In [ ]:
nmf_lab_matrix.shape

In [ ]:
plt.plot(range(0,23),s)
plt.title('Scree Plot of Singular Values: NMF Lab Data')
plt.xticks(range(0,23))
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')

In [ ]:
calculateKnee(s)

In [ ]:
model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(nmf_lab_matrix)
H = model.components_

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.heatmap(lab_matrix)

In [ ]:
ax = sns.heatmap(W)

In [ ]:
ax = sns.heatmap(H)

### Maternity and Ventilation Analysis

Given literature, it seemed beneficial to conduct a targeted cohort analysis. I will apply a similar process as performed above on my new dataset: Maternity and Ventilation patients. This dataset includes lab data for patients who had Maternity and ventilation procedure codes.

In [ ]:
maternity_df = pd.read_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/maternity_df.csv", low_memory = True)

In [ ]:
maternity_matrix = maternity_df.values

In [ ]:
maternity_df.head()

In [ ]:
#apply numpy SVD function to master matrix
maternity_U,maternity_s,maternity_V = np.linalg.svd(maternity_matrix, full_matrices = False)

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
maternity_svd = TruncatedSVD(n_components = 2)
maternity_svd.fit(maternity_matrix)
matSVD_results = maternity_svd.transform(maternity_matrix)

In [ ]:
#perform kmeans on maternity data 
matSVD_df = pd.DataFrame(matSVD_results)
#K means Clustering 
clust_labels, cent = doKmeans(matSVD_df, 2)
kmeans = pd.DataFrame(clust_labels)
matSVD_df.insert((matSVD_df.shape[1]),'kmeans',kmeans)

In [ ]:
matSVD_matrix = matSVD_df.values

Now that we have clustered the SVD results, we will perform NMF on the maternity dataframe and see how our results differ

In [ ]:
mat_data = maternity_df.drop(columns = ['Unnamed: 0'])
mat_data = mat_data.drop(columns = ['dischtime'])
mat_data = mat_data.drop(columns = ['dob'])

In [ ]:
mat_matrix = mat_data.values

In [ ]:
mat_model = NMF(n_components=None, init='random', random_state=0)
mat_W = mat_model.fit_transform(mat_matrix)
mat_H = mat_model.components_

In [ ]:
ax = sns.heatmap(mat_data)

In [ ]:
ax = sns.heatmap(mat_W)

In [ ]:
ax = sns.heatmap(mat_H)

In [ ]:
matNMF_df = mat_W @ mat_H
matNMF_df = pd.DataFrame(matNMF_df)
#perform clustering 
clust_labels, cent = doKmeans(matNMF_df, 2)
kmeans = pd.DataFrame(clust_labels)
matNMF_df.insert((matNMF_df.shape[1]),'kmeans',kmeans)

## NMF and SVD Functions 

In [ ]:
def computeTruncatedSVD(A):
    #apply numpy SVD function to input matrix
    k = calculateK(A)
    svd = TruncatedSVD(n_components = k)
    svd.fit(A)
    results = svd.transform(A)
    return results

def calculateK(A):
    #compute basic svd to find K value
    U,s,V = np.linalg.svd(A,full_matrices = False)
    k = calculateKnee(s)
    return k

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
def computeNMFComponents(A):
    #apply sklearn NMF function to input matrix
    k = calculateK(A)
    mat_model = NMF(n_components=k)
    mat_W = mat_model.fit_transform(A)
    mat_H = mat_model.components_
    return mat_W, mat_H

In [ ]:
def computeNMF(A):
   #apply sklearn NMF function to input matrix
    k = calculateK(A)
    mat_model = NMF(n_components=k)
    mat_W = mat_model.fit_transform(A)
    mat_H = mat_model.components_
    return mat_W @ mat_H

Lab Data Calculations

In [ ]:
labMatrix = lab_data.values
nmf_labMatrix = nmf_lab_data.values
sdd_labMatrix = sdd_lab_data.values

In [ ]:
svdLab_results = computeTruncatedSVD(labMatrix)
sddLab_results = computeTruncatedSVD(sdd_labMatrix)
nmfLab_results = computeNMF(nmf_labMatrix)

svdLab_resultsDF = pd.DataFrame(svdLab_results)
sddLab_resultsDF = pd.DataFrame(sddLab_results)
nmfLab_resultsDF = pd.DataFrame(nmfLab_results)

svdLab_resultsDF.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/svdLab_results.csv")
sddLab_resultsDF.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/sddLab_results.csv")
nmfLab_resultsDF.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/nmfLab_results.csv")

In [ ]:
nmfLabW, nmfLabH = computeNMFComponents(nmf_lab_matrix)

Master Matrix Calculations

In [ ]:
mmSVD_results = computeTruncatedSVD(master_matrix)
mmNMF_results = computeNMF(master_matrix)

mmSVD_resultsDF = pd.DataFrame(mmSVD_results)
mmNMF_resultsDF = pd.DataFrame(mmNMF_results)

mmSVD_resultsDF.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/mmSVD_results.csv")
mmNMF_resultsDF.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/mmNMF_results.csv")